# Robot Operation (BotOp) interface

This describes basics to interact with a real or simulated robot. The BotOp (=robot operation) interface is very narrow. The move methods set/overwrite a spline reference for the robot. (Also compliance around the reference can be set.) The gripper methods operate grippers. The getImage.. methods grab images or point clouds from the camera.

This interface different to a more *generic physical simulation* interface. If you're interested in the latter (e.g. to implement a gym environment) look at the `Simulation` tutorial. The simulation used here is a real-time threaded process that mimics the specific BotOp interface -- to make it swappable with a real robot.

The simulation can be run in many different modes: pure kinematic (no physics for objects), a physics simulator with physics for objects but still kinematic robot, a physic simulator with PD motors for the robot.

In [1]:
from robotic import ry
import numpy as np
import time

ry has global parameters, that can be defined in `rai.cfg` or with the following calls.
The simulation behaves very differently depending on `botim/engine` [physx or kinematic] and `multibody`

In [2]:
ry.params_add({'botsim/verbose': 2., 'physx/motorKp': 10000., 'physx/motorKd': 1000.})
ry.params_add({'botsim/engine': 'physx'}) #makes a big difference!
ry.params_add({'physx/multibody': True}) #makes a big difference!
ry.params_print()

-- ry.cpp:operator():99(0) python,
message: "Hello, the local 'rai.cfg' was loaded",
botsim/verbose: 2,
physx/motorKp: 10000,
physx/motorKd: 1000,
botsim/engine: physx,
physx/multibody: 1


In [3]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))
C.view(False, 'this is your workspace data structure C -- NOT THE SIMULTATION')

0

We open a robot interface in simulation (False). True would directly open communication to one or two pandas (depending no how many are defined in C). The `botsim/verbose` above leads to the explicit verbosity when creating the simulator interface.

In [4]:
bot = ry.BotOp(C, False)
#note that in sim, when physx multibody is activated, arms are going down! free floating...

-- kin_physx.cpp:PhysXInterface:768(0) starting PhysX engine ...
-- kin_physx.cpp:addGround:238(0) ... done starting PhysX engine
-- kin_physx.cpp:addGround:239(0) creating Configuration within PhysX ...
-- kin_physx.cpp:addLink:254(0) adding link 'world' as static with 1 shapes
 table
-- kin_physx.cpp:addMultiBody:466(0) adding multibody with base 'l_panda_base' with the following links ...
-- kin_physx.cpp:addMultiBody:486(0) adding multibody link 'l_panda_base' as kinematic with 1 shapes
 l_panda_link0_0
-- kin_physx.cpp:addMultiBody:486(0) adding multibody link 'l_panda_joint1' as dynamic with 1 shapes
 l_panda_link1_0
-- kin_physx.cpp:addMultiBody:486(0) adding multibody link 'l_panda_joint2' as dynamic with 2 shapes
 l_panda_link2_0 bellybutton
-- kin_physx.cpp:addMultiBody:486(0) adding multibody link 'l_panda_joint3' as dynamic with 1 shapes
 l_panda_link3_0
-- kin_physx.cpp:addMultiBody:486(0) adding multibody link 'l_panda_joint4' as dynamic with 1 shapes
 l_panda_link4_0
-- 

We define 2 reference poses, q0=home and q1=(2nd joint bend), so that we can move back and forth between them

In [5]:
qHome = bot.get_qHome()
q0 = qHome.copy()
q1 = q0.copy()
q1[1] = q1[1] + .2
print(q0, q1)

[ 0.  -0.5  0.  -2.   0.   2.  -0.5] [ 0.  -0.3  0.  -2.   0.   2.  -0.5]


The `moveTo` is the simplest way to move the robot from current to target. It internally creates a spline to the target with optimal timing and follows it. The call is *non-blocking*. Also, your workspace config C is not kept in sync with the real/sim. If you want to wait till the motion is finished, you need to do manually checking the /time_til_end_of_reference_spline/, and meanwhile staying sync'ed.

In [6]:
bot.moveTo(q1)

while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

The internal spline reference can be appended: As `moveTo` is non-blocking, you can append several moves like this:

In [7]:
print('timeToEnd:', bot.getTimeToEnd())
bot.moveTo(q0)
print('timeToEnd:', bot.getTimeToEnd())
bot.moveTo(q1)
print('timeToEnd:', bot.getTimeToEnd())
bot.moveTo(q0)

while bot.getTimeToEnd()>0:
    bot.sync(C, .1)

timeToEnd: -0.073524058876411
timeToEnd: 1.0957189420649807
timeToEnd: 2.191437884129961


Setting splines becomes reactive, when we can smoothly overwrite the spline reference with high frequency. Let's create a randomly moving target object and track it.

In [8]:
#this reference frame only appears in your workspace C - not the simulation!
target = C.addFrame('target', 'table')
target.setShape(ry.ST.marker, [.1])
target.setRelativePosition([0., .3, .3])
pos = target.getPosition()
cen = pos.copy()
C.view()

0

In [9]:
# you'll learn about KOMO later - this defines a basic Inverse Kinematics method
def IK(C, pos):
    q0 = C.getJointState()
    komo = ry.KOMO(C, 1, 1, 0, False) #one phase one time slice problem, with 'delta_t=1', order=0
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0) #cost: close to 'current state'
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome) #cost: close to qHome
    komo.addObjective([], ry.FS.positionDiff, ['l_gripper', 'target'], ry.OT.eq, [1e1]) #constraint: gripper position
    
    ret = ry.NLP_Solver(komo.nlp(), verbose=0) .solve()
    
    return [komo.getPath()[0], ret]

The following is just 'setting' the workspace C to the IK solution - no motion send to the real/robot:

In [10]:
for t in range(20):
    time.sleep(.1)
    pos = cen + .98 * (pos-cen) + 0.02 * np.random.randn(3)
    target.setPosition(pos)
    
    q_target, ret = IK(C, pos)
    print(ret)
    C.setJointState(q_target)
    C.view()

{ time: 0.005057, evals: 6, done: 1, feasible: 1, sos: 0.00789171, f: 0, ineq: 0, eq: 0.00138392 }
{ time: 0.001435, evals: 4, done: 1, feasible: 1, sos: 0.0055078, f: 0, ineq: 0, eq: 0.00038073 }
{ time: 0.001453, evals: 3, done: 1, feasible: 1, sos: 0.00485786, f: 0, ineq: 0, eq: 0.0031066 }
{ time: 0.000382, evals: 3, done: 1, feasible: 1, sos: 0.00552496, f: 0, ineq: 0, eq: 0.00285066 }
{ time: 0.000871, evals: 4, done: 1, feasible: 1, sos: 0.00481289, f: 0, ineq: 0, eq: 0.000176548 }
{ time: 0.000181, evals: 3, done: 1, feasible: 1, sos: 0.00416384, f: 0, ineq: 0, eq: 0.00404598 }
{ time: 0.001811, evals: 3, done: 1, feasible: 1, sos: 0.00394648, f: 0, ineq: 0, eq: 0.00118522 }
{ time: 0.000637, evals: 3, done: 1, feasible: 1, sos: 0.00410849, f: 0, ineq: 0, eq: 0.00429565 }
{ time: 0.001591, evals: 4, done: 1, feasible: 1, sos: 0.00454469, f: 0, ineq: 0, eq: 0.000211266 }
{ time: 0.000784, evals: 3, done: 1, feasible: 1, sos: 0.00470588, f: 0, ineq: 0, eq: 0.00357698 }
{ time: 0.

We now generate reative motion by smoothly overwriting the spline reference. Increasing time cost makes it more agressive (penalized total duration of estimated cubic spline).

In [11]:
for t in range(100):
    bot.sync(C, .1) #keep the workspace C sync'ed to real/sim, and idle .1 sec
    pos = cen + .98 * (pos-cen) + 0.02 * np.random.randn(3)
    target.setPosition(pos)
    
    q_target, ret = IK(C, pos)
    bot.moveTo(q_target, timeCost=5., overwrite=True)

Good practise is to always allow a user aborting motion execution. In this example, key 'q' will break the loop and call a home() (which is the same as moveTo(qHome, 1., True)

In [ ]:
for t in range(5):
    bot.moveTo(q1)
    bot.wait(C) #same as 'loop sync til keypressed or endOfTime', but also raises user window
    if bot.getKeyPressed()==ord('q'):
        break;
        
    bot.moveTo(q0)
    bot.wait(C)
    if bot.getKeyPressed()==ord('q'):
        break;

bot.home(C)

gripper movements also do not block:

In [ ]:
bot.gripperMove(ry._left, width=.02)

while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

bot.gripperMove(ry._left, width=.075)

while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

Always close the bot/sim properly:

In [ ]:
del bot
del C

As a side note, we can always check which global config parameters have been queried by the code so far. That gives an idea of which global parameters exist:

In [ ]:
ry.params_print()